# Is Julia fast?

Julia isn't fast *per se*.

One can write terribly slow code in any language, including Julia.

So let's ask a different question.

# *Can* Julia be fast?

 ### Microbenchmarks
 <img src="../playground/julia-microbench/benchmarks.svg" alt="drawing" width="800"/>

### More realistic case: Vandermonde matrix
(taken from [Steve's Julia intro](https://web.mit.edu/18.06/www/Fall17/1806/julia/Julia-intro.pdf))

[Vandermonde matrix:](https://en.wikipedia.org/wiki/Vandermonde_matrix)
\begin{align}V=\begin{bmatrix}1&\alpha _{1}&\alpha _{1}^{2}&\dots &\alpha _{1}^{n-1}\\1&\alpha _{2}&\alpha _{2}^{2}&\dots &\alpha _{2}^{n-1}\\1&\alpha _{3}&\alpha _{3}^{2}&\dots &\alpha _{3}^{n-1}\\\vdots &\vdots &\vdots &\ddots &\vdots \\1&\alpha _{m}&\alpha _{m}^{2}&\dots &\alpha _{m}^{n-1}\end{bmatrix}\end{align}

In [3]:
using PyCall

┌ Info: Recompiling stale cache file C:\Users\carsten\.julia\compiled\v1.1\PyCall\GkzkC.ji for PyCall [438e738f-606a-5dbb-bf0a-cddfbfd45ab0]
└ @ Base loading.jl:1184


In [4]:
np = pyimport("numpy")

PyObject <module 'numpy' from 'C:\\Users\\carsten\\Anaconda3\\lib\\site-packages\\numpy\\__init__.py'>

In [6]:
np.vander(1:5, increasing=true)

5×5 Array{Int32,2}:
 1  1   1    1    1
 1  2   4    8   16
 1  3   9   27   81
 1  4  16   64  256
 1  5  25  125  625

The source code for this function is [here](https://github.com/numpy/numpy/blob/v1.16.1/numpy/lib/twodim_base.py#L475-L563). It calls `np.multiply.accumulate` which is implemented in C [here](https://github.com/numpy/numpy/blob/deea4983aedfa96905bbaee64e3d1de84144303f/numpy/core/src/umath/ufunc_object.c#L3678). However, this code doesn't actually perform the computation, it basically only checks types and stuff. The actual kernel that gets called is [here](https://github.com/numpy/numpy/blob/deea4983aedfa96905bbaee64e3d1de84144303f/numpy/core/src/umath/loops.c.src#L1742). This isn't even C code but a template for C code which is used to generate type specific kernels.

Overall, this setup only supports a limited set of types, like `Float64`, `Float32`, and so forth.

Here is a simple Julia implementation

In [7]:
function vander(x::AbstractVector{T},  n=length(x)) where T
    m = length(x)
    V = Matrix{T}(undef, m, n)
    for j = 1:m
        V[j,1] = one(x[j])
    end
    for i= 2:n
        for j = 1:m
            V[j,i] = x[j] * V[j,i-1]
            end
        end
    return V
end

vander (generic function with 2 methods)

In [8]:
vander(1:5)

5×5 Array{Int64,2}:
 1  1   1    1    1
 1  2   4    8   16
 1  3   9   27   81
 1  4  16   64  256
 1  5  25  125  625

#### A quick speed comparison

<details>
  <summary>Show Code</summary>
<br>
    
```julia
using BenchmarkTools, Plots
ns = exp10.(range(1, 4, length=30));

tnp = Float64[]
tjl = Float64[]
for n in ns
    x = 1:n |> collect
    push!(tnp, @belapsed np.vander(\$x) samples=3 evals=1)
    push!(tjl, @belapsed vander(\$x) samples=3 evals=1)
end
plot(ns, tnp./tjl, m=:circle, xscale=:log10, xlab="matrix size", ylab="NumPy time / Julia time", legend=:false)
```
</details>

 <img src="../playground/vandermonde/vandermonde.svg" alt="drawing" width="600"/>

Note that the clean and concise Julia implementation is **beating numpy's C implementation for small matrices** and is **on-par for large matrix sizes**.

At the same time, the Julia code is generic and works for arbitrary types!

In [52]:
vander(Int32[4, 8, 16, 32])

4×4 Array{Int32,2}:
 1   4    16     64
 1   8    64    512
 1  16   256   4096
 1  32  1024  32768

It even works for non-numerical types. The only requirement is that the type has a *one* (identity element) and a multiplication operation defined.

In [47]:
vander(["this", "is", "a", "test"])

4×4 Array{String,2}:
 ""  "this"  "thisthis"  "thisthisthis"
 ""  "is"    "isis"      "isisis"      
 ""  "a"     "aa"        "aaa"         
 ""  "test"  "testtest"  "testtesttest"

Here, `one(String) == ""` since the empty string is the identity under multiplication (string concatenation).

# How can Julia be fast?

<p><img src="../presentation/images/from_source_to_native.png" alt="drawing" width="800"/></p>
 
**AST = Abstract Syntax Tree**

**[LLVM](https://de.wikipedia.org/wiki/LLVM) = Low Level Virtual Machine**

### Specialization and code inspection

Julia specializes on the types of function arguments. 

When a function is called for the first time, Julia compiles efficient machine code for the given input types.

If it is called again, the already existing machine code is reused, until we call the function with different input types.

In [30]:
func(x,y) = x^2 + y

func (generic function with 1 method)

In [31]:
@time func(1,2)
@time func(1,2)

  0.004314 seconds (1.97 k allocations: 110.536 KiB)
  0.000001 seconds (4 allocations: 160 bytes)


3

**First call:** compilation + running the code

**Second call:** running the code

In [32]:
@time func(1,2)

  0.000005 seconds (4 allocations: 160 bytes)


3

If the input types change, Julia compiles a new specialization of the function.

In [33]:
@time func(1.3,4.8)
@time func(1.3,4.8)

  0.005138 seconds (1.95 k allocations: 109.212 KiB)
  0.000002 seconds (5 allocations: 

6.49

176 bytes)


We now have two efficient codes, one for all `Int64` inputs and another one for all `Float64` arguments, in the cache.

### *But I really want to see what happens!*

We can inspect the code at all transformation stages with a bunch of macros:

* The AST after parsing (**`@macroexpand`**)
* The AST after lowering (**`@code_typed`**, **`@code_warntype`**)
* The AST after type inference and optimization (**`@code_lowered`**)
* The LLVM IR (**`@code_llvm`**)
* The assembly machine code (**`@code_native`**)

In [38]:
@code_typed func(1,2)

CodeInfo(
1 ─ %1 = (Base.mul_int)(x, x)::Int64
│   %2 = (Base.add_int)(%1, y)::Int64
└──      return %2
) => Int64

In [39]:
@code_lowered func(1,2)

CodeInfo(
1 ─ %1 = (Core.apply_type)(Base.Val, 2)
│   %2 = (%1)()
│   %3 = (Base.literal_pow)(Main.:^, x, %2)
│   %4 = %3 + y
└──      return %4
)

In [42]:
@code_llvm func(1,2)


;  @ In[30]:1 within `func'
; Function Attrs: uwtable
define i64 @julia_func_13845(i64, i64) #0 {
top:
; ┌ @ intfuncs.jl:243 within `literal_pow'
; │┌ @ int.jl:54 within `*'
    %2 = mul i64 %0, %0
; └└
; ┌ @ int.jl:53 within `+'
   %3 = add i64 %2, %1
; └
  ret i64 %3
}


We can remove the comments (lines starting with `;` using `debuginfo=:none`).

In [46]:
@code_llvm debuginfo=:none func(1,2)


; Function Attrs: uwtable
define i64 @julia_func_13845(i64, i64) #0 {
top:
  %2 = mul i64 %0, %0
  %3 = add i64 %2, %1
  ret i64 %3
}


In [49]:
@code_native func(1,2)

	.text
; ┌ @ In[30]:1 within `func'
	pushq	%rbp
	movq	%rsp, %rbp
; │┌ @ intfuncs.jl:243 within `literal_pow'
; ││┌ @ int.jl:54 within `*'
	imulq	%rcx, %rcx
; │└└
; │┌ @ int.jl:53 within `+'
	leaq	(%rcx,%rdx), %rax
; │└
	popq	%rbp
	retq
	nop
; └


Let's compare this to `Float64` input.

In [50]:
@code_native func(1.2,2.9)

	.text
; ┌ @ In[30]:1 within `func'
	pushq	%rbp
	movq	%rsp, %rbp
; │┌ @ intfuncs.jl:243 within `literal_pow'
; ││┌ @ float.jl:399 within `*'
	vmulsd	%xmm0, %xmm0, %xmm0
; │└└
; │┌ @ float.jl:395 within `+'
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
	popq	%rbp
	retq
	nop
; └


## How important is code specialization?

Let's try to estimate the performance gain by specialization.

We wrap our numbers into a custom type which internally stores them as `Any` to prevent specialization.

(This is qualitatively comparable to what Python does.)

In [52]:
struct Anything
    value::Any
end

operation(x::Number) = x^2 + sqrt(x)
operation(x::Anything) = x.value^2 + sqrt(x.value)

operation (generic function with 2 methods)

In [64]:
@btime operation(2);
@btime operation(2.0);

x = Anything(2.0)
@btime operation($x);

  1.299 ns (0 allocations: 0 bytes)
  1.299 ns (0 allocations: 0 bytes)
  50.861 ns (3 allocations: 48 bytes)


**That's about an 40 times slowdown!**

In [81]:
@code_native operation(2.0)

	.text
; ┌ @ In[52]:5 within `operation'
; │┌ @ math.jl:492 within `sqrt'
; ││┌ @ In[52]:5 within `<'
	vxorps	%xmm1, %xmm1, %xmm1
	vucomisd	%xmm0, %xmm1
; ││└
	ja	L23
; │└
; │┌ @ intfuncs.jl:243 within `literal_pow'
; ││┌ @ float.jl:399 within `*'
	vmulsd	%xmm0, %xmm0, %xmm1
; │└└
; │┌ @ math.jl:493 within `sqrt'
	vsqrtsd	%xmm0, %xmm0, %xmm0
; │└
; │┌ @ float.jl:395 within `+'
	vaddsd	%xmm0, %xmm1, %xmm0
; │└
	retq
L23:
	pushq	%rbp
	movq	%rsp, %rbp
; │ @ In[52]:5 within `operation'
; │┌ @ math.jl:492 within `sqrt'
	subq	$32, %rsp
	movabsq	$throw_complex_domainerror, %rax
	movl	$71726320, %ecx         # imm = 0x44674F0
	vmovaps	%xmm0, %xmm1
	callq	*%rax
	ud2
	ud2
	nopl	(%rax,%rax)
; └└


In [79]:
@code_native operation(x)

	.text
; ┌ @ In[52]:6 within `operation'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%rsi
	pushq	%rdi
	pushq	%rbx
	andq	$-32, %rsp
	subq	$128, %rsp
	movq	%rdx, %rdi
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 64(%rsp)
	movq	%rdi, 96(%rsp)
	movabsq	$jl_get_ptls_states, %rax
	vzeroupper
	callq	*%rax
	movq	%rax, %rsi
	movq	$4, 64(%rsp)
	movq	(%rsi), %rax
	movq	%rax, 72(%rsp)
	leaq	64(%rsp), %rax
	movq	%rax, (%rsi)
	movq	(%rdi), %rdi
; │┌ @ sysimg.jl:18 within `getproperty'
	movq	(%rdi), %rax
; │└
	movabsq	$jl_system_image_data, %rcx
	movq	%rcx, 32(%rsp)
	movabsq	$jl_system_image_data, %rcx
	movq	%rcx, 40(%rsp)
	movq	%rax, 48(%rsp)
	movabsq	$jl_system_image_data, %rax
	movq	%rax, 56(%rsp)
	movabsq	$jl_apply_generic, %r15
	leaq	32(%rsp), %r14
	movl	$4, %edx
	movq	%r14, %rcx
	callq	*%r15
	movq	%rax, %rbx
; │┌ @ sysimg.jl:18 within `getproperty'
	movq	(%rdi), %rax
	movq	%rbx, 88(%rsp)
; │└
	movabsq	$jl_system_image_data, %rcx
	movq	%rcx, 32(%rsp)
	movq	%rax, 40(%rsp)
	movl	$2,

# Are explicit type annotations necessary? (like in C or Fortran)

Note that Julia's type inference is powerful. Specifying types **is not** necessary for best performance!

In [76]:
function my_function(x)
    y = rand()
    z = rand()
    x+y+z
end

function my_function_typed(x::Int)::Float64
    y::Float64 = rand()
    z::Float64 = rand()
    x+y+z
end

my_function_typed (generic function with 1 method)

In [77]:
@btime my_function(10);
@btime my_function_typed(10);

  6.600 ns (0 allocations: 0 bytes)
  6.500 ns (0 allocations: 0 bytes)


 However, annotating types explicitly can serve a purpose.

* **Define a user interface/type filter** (will throw error if incompatible type is given)
* Enforce conversions
* Rarely, help the compiler infer types in tricky situations